In [1]:
# setup cell!!

from canvasapi import Canvas
from canvasapi.exceptions import BadRequest
from datetime import datetime, time, timedelta
from zoneinfo import ZoneInfo
import ipywidgets as widgets
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd
from thefuzz import fuzz, process

# setup canvas connection
tokenfile = open("canvas-token", "r")
token = tokenfile.readline()
tokenfile.close()
url = "https://westminster.instructure.com/"
canvas = Canvas(url, token)

# now i'ma hardcode the courses that people will ask for extensions in
# this will avoid a number of (slow!) requests and filters.
math202id = 3387585
math202 = canvas.get_course(math202id)
phys309id = 3388078
phys309 = canvas.get_course(phys309id)
coursedict = {
    "MATH 202": math202,
    "PHYS 309": phys309
}

# setup google connection
# If modifying these scopes, delete the file google-token.json.
scopes = ['https://www.googleapis.com/auth/spreadsheets.readonly']
spreadsheet_id = '1mtc172TQIju8BwAiqwFS8Fwtz1I8ZN6Aw7SwfrVvdG0'
data_range = 'A2:I' #include the A column for a unique timestamp

creds = None
# The file google-token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first time.
# I am .gitignoring both google-token.json and google-credentials.json.
if os.path.exists('google-token.json'):
    creds = Credentials.from_authorized_user_file('google-token.json', scopes)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'google-credentials.json', scopes)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('google-token.json', 'w') as token:
        token.write(creds.to_json())


In [2]:
# utility function: combine two PaginatedLists
def combine_pl(pl1, pl2):
    # iterate through both to build the elements lists
    # this is because these objects are loaded lazily from the API
    for item in pl1: pass
    for item in pl2: pass
    pl1._elements = pl1._elements + pl2._elements
    return pl1

In [3]:
# utility function: remove unpublished assignments
def remove_unpublished(pl):
    i = 0
    dellist = []
    for item in pl: 
        # keep track of not-published items
        if not item.published: dellist.append(i)
        i+=1
    # remove assignments that aren't published
    for i in sorted(dellist, reverse=True):
        del pl._elements[i]
    return pl

In [4]:
# I will also pre-pull the assignment lists and do a little pre-filtering.
# in math 202 specifically, I only care about problem sets and webwork
# so let's grab those by groups
for group in math202.get_assignment_groups(): 
    if group.name == "Problem Sets":
        math202_assgs = math202.get_assignments_for_group(group.id)
    elif group.name == "Webwork":
        math202_webwork = math202.get_assignments_for_group(group.id)

math202_assgs = remove_unpublished(combine_pl(math202_assgs, math202_webwork))

phys309_assgs = remove_unpublished(phys309.get_assignments())


In [5]:
# read data from spreadsheet
try:
    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet_id,
                                range=data_range).execute()
    requests = result.get('values', [])
    requestsdf = pd.DataFrame(requests, columns = ["timestamp", "name", "email", "classname", 
        "assignment", "due date", "need1", "need2", "status"])
    # pull out the ones that I need to do something with
    # tododf = requestsdf[pd.isna(requestsdf["status"])]
    tododf = requestsdf
    

    if not requests:
        print('No data found.')
except HttpError as err:
    print(err)

In [6]:
# utility function: build dropdown list of assignments (no fuzz)
def find_assignment(assglist):
    opts = [("Ignore this request", None)]
    for assg in assglist:
        opts.append((assg.name, assg))
    dropdown = widgets.Dropdown(
        options=opts,
        value=None,
        description="Assignment: ",
        disabled=False,
    )
    return dropdown

In [7]:
# DEP -- utility function: fuzzy string match assignments
# given a list of assignments and a string that a student wrote,
# I'm going to create a dropdown of possibilities,
# sorted by matching score,
# and pre-populated with my best guess.
# Currently I am deprecating this because it needs a little massaging.
def fuzzy_find_assignment(assglist, inputstr):
    namesdict = {}
    for assg in assglist:
        namesdict[assg.name]=assg
    # calculate and sort by matching score: 
    fuzzresults = process.extract(inputstr, namesdict.keys())
    opts = [] 
    for result in fuzzresults:
        opts.append( (result[0], namesdict[result[0]]) )
        # this is now a list of tuples like widgets.Dropdown wants.
    dropdown = widgets.Dropdown(
        options=opts,
        # value=None, -- I'm going to prepopulate with my best guess.
        description="Probably: ",
        disabled=False,
    )
    return dropdown

In [9]:
output = widgets.Output()
assignmentdict = {} #This is going to record what I click, and then we merge on timestamp.
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

def buttonclick(timestamp, dropdown):
    assignmentdict[timestamp.value] = dropdown.value #oops
    with output:
        print(timestamp.value + " " + str(dropdown.value))

def requestbox(request): 
    timestamp = widgets.Label(value=request["timestamp"])
    name = widgets.Label(value=request["name"])
    dropdown = find_assignment(math202_assgs)
    button = widgets.Button(description = "Looks good!")
    button.on_click(lambda b: buttonclick(timestamp, dropdown))

    return widgets.VBox([
        name,
        widgets.HBox([widgets.Label(value=request["assignment"]), 
                      dropdown]),
        widgets.Label(value=request["due date"]), 
        widgets.HTML(value=request["need1"]),
        widgets.HTML(value=request["need2"]),
        button
        ])
tab = widgets.Tab(children = tododf.apply(requestbox, axis=1).tolist())
display(tab, output)

Output()

In [10]:
tododf.merge(pd.DataFrame(assignmentdict), how='left', on='timestamp')
print(tododf)

ValueError: If using all scalar values, you must pass an index

In [9]:
def build_override_dict(request): # request is a row of the google sheet
    [timestamp, name, email, coursename, assignment, dd, need1, need2, status] = request

    # find the correct course and lookup the student
    course = coursedict[coursename]
    try: student = course.get_users(search_term = email)[0]
    except IndexError as err:
        print("haha that email's wrong")
        return
    
    # parse the date correctly as a datetime object
    # comes in from google forms as, e.g., "1/30"
    duetime = timedelta(hours=23,minutes=59)
    duedate = datetime.strptime(dd+"/23", '%m/%d/%y').replace(tzinfo=ZoneInfo("America/Denver"))
    duedate = duedate + duetime

    return {
        'student_ids': [student.id],
        'due_at': duedate, 
        'all_day': True,
        'all_day_date':  str(duedate.date())
    }

# a thought: should this actually be able to deal with multiple students requesting the same new due date for the same assignment?
# that may be more headache than it is actually worth.
# maybe what should happen is, I build individual override dicts,
# and then before I fire off all the overrides,
# a helper function checks *all* the override dicts I have built,
# together with all existing overrides,
# and collapses any that it can.

In [10]:
# so now what I am going to do is something like, 
# filter the requestsdf to only have the todo ones
# then tododf.apply(build_override_dict, axis=1) 

# at some point we also need to do the assignment lookup
# and finally commit all of those overrides to canvas.

In [11]:
# hey so this is 74% of the body of the write override function
# i wanna write this as assg.write_override()
def write_override(self, ovdict):
    overrides = self.get_overrides()
    for override in overrides:
        # if there is some kind of intersection between the student ids in the existing overrides
        # and the student id in the ovdict I am throwing you, 
        # then we should edit the existing override to remove the student
        # and create a new override for the student

    try:
        self.create_override(assignment_override = dict)
    except BadRequest: # I need to be a little more careful here.
        # It's possible that the student is *one* of several students in an override.
        # If so, I need to remove them from the old override
        # and create them a new override
        # or else add them to an existing override
        # (otherwise it might fuck up other people's overrides!)
        bad_override = next((ov for ov in test_assg.get_overrides() if 12261567 in ov.student_ids), None)
        bad_override.edit(
            assignment_override={
                'student_ids': [12261567],
                'due_at': datetime(2023,2,1,23,59,59,tzinfo=ZoneInfo('America/Denver')),
                'all_day': True,
                'all_day_date':  '2023-02-01'
            }
        )


IndentationError: expected an indented block after 'for' statement on line 5 (2799972295.py, line 11)